## <font color='purple'>Books Recommender System Using Clustering | Colaborative Based Filtering</font>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
books = pd.read_csv('datasets/BX-Books.csv',on_bad_lines='skip',encoding='latin-1',sep=';')

In [ ]:
books.head(5)

In [ ]:
books.shape

In [ ]:
books.columns

In [ ]:
books=books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [ ]:
books

In [ ]:
books.rename(
    columns={
        'Book-Title' : 'title',
        'Book-Author' : 'author',
        'Year-Of-Publication' : 'year',
        'Publisher' : 'publisher',
        'Image-URL-L' : 'image_url'
    },inplace=True
)

In [ ]:
books.head(2)

In [ ]:
users = pd.read_csv('datasets/BX-Users.csv',sep=';',on_bad_lines='skip',encoding='latin-1')

In [ ]:
users.head(5)

In [ ]:
users.shape

In [ ]:
ratings=pd.read_csv('datasets/BX-Book-Ratings.csv',sep=';',on_bad_lines='skip',encoding='latin-1')

In [ ]:
ratings.head(5)

In [ ]:
ratings.rename(columns={
    'User-ID' : 'user_id',
    'Book-Rating' : 'rating'
},inplace=True)

In [ ]:
ratings.head(5)

In [ ]:
ratings.shape

In [ ]:
print(books.shape)
print(users.shape)
print(ratings.shape)

In [ ]:
ratings['user_id'].value_counts()

In [ ]:
ratings['user_id'].unique().shape

In [ ]:
x=ratings['user_id'].value_counts()>200

In [ ]:
x[x].shape

In [ ]:
y = x[x].index

In [ ]:
y

In [ ]:
ratings=ratings[ratings['user_id'].isin(y)]

In [ ]:
ratings.head(5)

In [ ]:
ratings.shape

In [ ]:
ratings_with_books =ratings.merge(books,on='ISBN')

In [ ]:
ratings_with_books.head(5)

In [ ]:
num_rating =ratings_with_books.groupby('title')['rating'].count().reset_index()

In [ ]:
num_rating.head()

In [ ]:
num_rating.shape

In [ ]:
num_rating.rename(columns={'rating' : 'num_of_rating'},inplace=True)

In [ ]:
num_rating.head()

In [ ]:
final_rating=ratings_with_books.merge(num_rating,on='title')

In [ ]:
final_rating.head()

In [ ]:
final_rating.shape

In [ ]:
final_rating=final_rating[final_rating['num_of_rating']>=50]

In [ ]:
final_rating.head()

In [ ]:
final_rating.sample(10)

In [ ]:
final_rating.shape

In [ ]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [ ]:
final_rating.shape

In [ ]:
final_rating.head()

In [ ]:
book_pivot = final_rating.pivot_table(index='title',columns='user_id',values='rating')

In [ ]:
book_pivot

In [ ]:
book_pivot.shape

In [ ]:
book_pivot.fillna(0,inplace=True)

In [ ]:
book_pivot

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
book_sparse = csr_matrix(book_pivot)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [ ]:
model.fit(book_sparse)

In [ ]:
distance , suggestion = model.kneighbors(book_pivot.iloc[344,:].values.reshape(1,-1),n_neighbors=6)

In [ ]:
distance

In [ ]:
suggestion

In [ ]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

In [ ]:
books_name = book_pivot.index

In [ ]:
books_name

In [ ]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(books_name,open('artifacts/books_name.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))

In [ ]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance , suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)

    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [ ]:
book_name="You Belong To Me"
recommend_book(book_name)